In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds
from tensorflow import keras

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.list_physical_devices("GPU") else "NOT AVAILABLE")

In [ ]:
trainset = pd.read_csv('../input/covid-19-nlp-text-classification/Corona_NLP_train.csv')
test = pd.read_csv('../input/covid-19-nlp-text-classification/Corona_NLP_test.csv')

In [ ]:
trainset['Sentiment'].unique()

In [ ]:
trainset['Sentiment'].value_counts().plot.pie()

In [ ]:
trainset['Sentiment'].replace(['Extremely Negative', 'Extremely Positive'], ['Negative', 'Positive'], inplace=True)
test['Sentiment'].replace(['Extremely Negative', 'Extremely Positive'], ['Negative', 'Positive'], inplace=True)

In [ ]:
trainset['Sentiment'].value_counts().plot.pie()

In [ ]:
trainset['Sentiment'] = LabelEncoder().fit_transform(trainset['Sentiment'])
test['Sentiment'] = LabelEncoder().fit_transform(test['Sentiment'])

In [ ]:
train, val = train_test_split(trainset, test_size=0.2, random_state=42)

In [ ]:
x_train = train['OriginalTweet']
x_val = val['OriginalTweet']
x_test = test['OriginalTweet']
y_train = train['Sentiment']
y_val = val['Sentiment']
y_test = test['Sentiment']

In [ ]:
x_train_ts = tf.constant(x_train, dtype = tf.string, shape=[32925,])
x_val_ts = tf.constant(x_val, dtype = tf.string, shape=[8232,])
x_test_ts = tf.constant(x_test, dtype = tf.string, shape=[3798,])
y_train_ts = tf.constant(y_train, dtype = tf.float32, shape=[32925,])
y_val_ts = tf.constant(y_val, dtype = tf.float32, shape=[8232,])
y_test_ts = tf.constant(y_test, dtype = tf.float32, shape=[3798,])

In [ ]:
# encode text with universal-sentence-encoder-large
embedding = "https://tfhub.dev/google/universal-sentence-encoder-large/5"
hub_layer = hub.KerasLayer(embedding, input_shape=[], 
                           dtype=tf.string, trainable=True)

In [ ]:
model = tf.keras.Sequential()
model.add(hub_layer)

model.add(tf.keras.layers.Dropout(0.3)),
model.add(tf.keras.layers.BatchNormalization()),

model.add(tf.keras.layers.Dense(180, kernel_regularizer='l1_l2', activation='relu'))
model.add(tf.keras.layers.Dropout(0.3)),
model.add(tf.keras.layers.BatchNormalization()),

model.add(tf.keras.layers.Dense(180, kernel_regularizer='l1_l2', activation='relu'))
model.add(tf.keras.layers.Dropout(0.3)),
model.add(tf.keras.layers.BatchNormalization()),

model.add(tf.keras.layers.Dense(180, kernel_regularizer='l1_l2',  activation='relu'))
model.add(tf.keras.layers.Dropout(0.3)),
model.add(tf.keras.layers.BatchNormalization()),

model.add(tf.keras.layers.Dense(3, kernel_regularizer='l1_l2', activation='softmax'))

model.summary()

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0002)
model.compile(optimizer=optimizer,
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

In [ ]:
callbacks = keras.callbacks.EarlyStopping(
    patience = 10,
    min_delta = 0.01,
    restore_best_weights = True,
)

In [ ]:
history = model.fit(x_train_ts, y_train_ts,
                    epochs = 600,
                    validation_data = (x_val_ts, y_val_ts),
                    batch_size = 512,
                    callbacks = [callbacks],
                    verbose = 0)

In [ ]:
results = model.evaluate(x_test_ts, y_test_ts, batch_size = 512, verbose=2)

for name, value in zip(model.metrics_names, results):
  print("%s: %.4f" % (name, value))

In [ ]:
history_dict = history.history

acc = history_dict['accuracy']
val_acc = history_dict['val_accuracy']
loss = history_dict['loss']
val_loss = history_dict['val_loss']

epochs = range(1, len(acc) + 1)

# "bo" is for "blue dot"
plt.plot(epochs, loss, 'bo', alpha=0.4, label='Training loss')
# b is for "solid blue line"
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

In [ ]:
plt.plot(epochs, acc, 'bo', alpha=0.4, label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')

plt.show()